In [ ]:
import torch
from torch import nn
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
#model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True)
model.eval()

In [ ]:
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]


from PIL import Image
from torchvision import transforms
input_image = Image.open("collegeville_bakery.jpg")
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x[0:3, :, :]),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
X = preprocess(input_image)
X_orig = X.clone()

In [ ]:
y = torch.zeros(1000)
## TODO: Make y a 1-hot vector for what we're trying to trick

# Require gradient on image
X = X.requires_grad_()
# Disable gradient in imagenet
for p in model.parameters():
    p.requires_grad_(False)

loss_fn = nn.BCEWithLogitsLoss()

optimizer = torch.optim.Adam([X], lr=1e-3)
lam = 10
n_iters = 1000
losses = []
sigmoid_scale = 3
for i in range(n_iters):
    optimizer.zero_grad()
    mu = torch.mean(torch.flatten(X, 1, 2), dim=1)
    XNorm = X - mu.view(3, 1, 1)
    std = torch.std(torch.flatten(XNorm, 1, 2), dim=1)
    std = std.view(3, 1, 1)
    XNorm = XNorm / std

    y_est = model(XNorm.unsqueeze(0)).flatten()
    loss = loss_fn(y_est, y)
    loss.backward()
    optimizer.step()
    if i%10 == 0:
        print(loss.item())
        probabilities = torch.softmax(y_est, 0)
        top5_prob, top5_catid = torch.topk(probabilities, 5)
        for i in range(top5_prob.size(0)):
            print(categories[top5_catid[i]], top5_prob[i].item())
        

In [ ]:
print(X-X_orig)